In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

Info    : Increasing process stack size (8192 kB < 16 MB)


In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000341279s, CPU 0.000343s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00576711s, CPU 0.005769s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 50%] Meshing curve 3 order 2
Info    : [ 70%] Meshing curve 4 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.781163
Info    : Done meshing order 2 (Wall 0.000696813s, CPU 0.000697s)
Info    : 357 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneStress)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 61)


LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357)

In [4]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
load = FEM.load("right", fx=1)

("right", 1, 0, 0)

In [5]:
u0 = FEM.initialDisplacement(problem, "body", ux=0, uy=0)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :u2D, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [6]:
FEM.initialDisplacement!("left", u0, ux=0, uy=0)
u0

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :u2D, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [7]:
v0 = FEM.initialVelocity(problem, "body", vx=0, vy=0)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :v2D, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [8]:
FEM.initialVelocity!("body", v0, vx=0, vy=0)

In [9]:
K = FEM.stiffnessMatrix(problem)
M = FEM.massMatrix(problem)
f = FEM.loadVector(problem, [load])

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :f2D, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [10]:
FEM.applyBoundaryConditions!(K, M, f, [supp])

In [11]:
Tmin = FEM.smallestPeriodTime(K, M)

┌ Warning: The error in the calculation of the largest eigenvalue is too large: 2.8616331443495193e9
└ @ LowLevelFEM /home/perebal/Dokumentumok/GitHub/LowLevelFEM.jl/src/linear.jl:2399


6.729882186080665e-16

In [12]:
u, v = FEM.CDM(K, M, f, u0, v0, 10Tmin, Tmin / π)

(LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … -9.863018604720215e-13 -3.119515388217235e-12; 0.0 0.0 … 1.9736983426493352e-12 6.281711664380167e-12], [0.0, 2.1421880326816568e-16, 4.2843760653633137e-16, 6.42656409804497e-16, 8.568752130726627e-16, 1.0710940163408283e-15, 1.285312819608994e-15, 1.4995316228771598e-15, 1.7137504261453255e-15, 1.927969229413491e-15  …  4.712813671899643e-15, 4.927032475167808e-15, 5.141251278435974e-15, 5.355470081704139e-15, 5.569688884972305e-15, 5.78390768824047e-15, 5.998126491508636e-15, 6.212345294776801e-15, 6.426564098044967e-15, 6.6407829013131324e-15], Int64[], 32, :u2D, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357)), LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … -3145

In [13]:
FEM.showDoFResults(u, :uvec, visible=true)

1

In [14]:
FEM.showDoFResults(v, :vvec)

2

In [15]:
e1 = FEM.solveStrain(u)

LowLevelFEM.TensorField([[0.0 0.0 … 2.1656649644394256e6 9.435730353950625e6; 0.0 0.0 … -1.6295270620268112e7 -7.06413591756844e7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -1.7464644351726726e7 -7.560837776779991e7], [0.0 0.0 … -147712.9267132491 -651001.9052613593; 0.0 0.0 … -786463.6153555952 -3.4107693340265425e6; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 777744.0477586671 3.399676263209797e6], [0.0 0.0 … -43072.20407786938 -139598.65356258227; 0.0 0.0 … -36998.220016823856 -120245.38692045017; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … 23136.791246666 75341.49280105352], [0.0 0.0 … -0.00034132755408595837 -0.0010468338331834036; 0.0 0.0 … -0.00021408770581666736 -0.000617684716226905; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -0.000186206314946572 -0.0005356467927994401], [0.0 0.0 … 778.7093263240268 2654.8934302450784; 0.0 0.0 … -1006.7660504090962 -3247.666441927787; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -600.4502025001691 -1875.020095193216], [0.0 0.0 … -7.397537179186804e-5 -0.00031646034501837765; 0.0 0.0 …

In [16]:
FEM.showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [17]:
e2 = FEM.solveStrain(u, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … -0.15744292613213204 -0.6707558419003363; 0.0 0.0 … 0.17982894550629724 0.7651409322379598; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -58338.5252966283 -188459.45487268452], [0.0, 2.1421880326816568e-16, 4.2843760653633137e-16, 6.42656409804497e-16, 8.568752130726627e-16, 1.0710940163408283e-15, 1.285312819608994e-15, 1.4995316228771598e-15, 1.7137504261453255e-15, 1.927969229413491e-15  …  4.712813671899643e-15, 4.927032475167808e-15, 5.141251278435974e-15, 5.355470081704139e-15, 5.569688884972305e-15, 5.78390768824047e-15, 5.998126491508636e-15, 6.212345294776801e-15, 6.426564098044967e-15, 6.6407829013131324e-15], Int64[], 32, :e, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [18]:
FEM.showDoFResults(e2, :e)

4

In [19]:
e3 = FEM.elementsToNodes(e1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … -0.15744292613213204 -0.6707558419003363; 0.0 0.0 … 0.08991447275314862 0.3825704661189799; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -58338.5252966283 -188459.45487268452], [0.0, 2.1421880326816568e-16, 4.2843760653633137e-16, 6.42656409804497e-16, 8.568752130726627e-16, 1.0710940163408283e-15, 1.285312819608994e-15, 1.4995316228771598e-15, 1.7137504261453255e-15, 1.927969229413491e-15  …  4.712813671899643e-15, 4.927032475167808e-15, 5.141251278435974e-15, 5.355470081704139e-15, 5.569688884972305e-15, 5.78390768824047e-15, 5.998126491508636e-15, 6.212345294776801e-15, 6.426564098044967e-15, 6.6407829013131324e-15], Int64[], 32, :e, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [20]:
FEM.showDoFResults(e3, :e)

5

In [21]:
s1 = FEM.solveStress(u)

LowLevelFEM.TensorField([[0.0 0.0 … 3.161122579002902e12 1.3698538252930828e13; 0.0 0.0 … -2.5069647108104785e12 -1.0867901411643754e13; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 1.2739515263917834e11 5.549981673076117e11; 0.0 0.0 … -1.2099440236239926e11 -5.247337436963912e11; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -1.0163301866416397e10 -3.299264096187829e10; 0.0 0.0 … -5.692033848742131e9 -1.8499290295453873e10; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -91.31892827479604 -285.6888387022043; 0.0 0.0 … -32.93657012564113 -95.02841788106231; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 6.621308479383464e8 2.1432892973543725e9; 0.0 0.0 … -1.548870846783225e8 -4.996409910658134e8; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … -18.682322817465824 -78.88550871935782; 0.0 0.0 … 7.557707003400467 33.09771469787636; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.03294278511050849 … -4.6479970229160887e8 -1.5324010229788737e9; 0.0 0.0

In [22]:
FEM.showStressResults(s1, :s)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


6

In [23]:
s2 = FEM.solveStress(u, DoFResults=true)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … -35314.8004995398 -150443.3809107587; 0.0 0.0 … 13832.995808176713 58856.99478753536; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 2.1421880326816568e-16, 4.2843760653633137e-16, 6.42656409804497e-16, 8.568752130726627e-16, 1.0710940163408283e-15, 1.285312819608994e-15, 1.4995316228771598e-15, 1.7137504261453255e-15, 1.927969229413491e-15  …  4.712813671899643e-15, 4.927032475167808e-15, 5.141251278435974e-15, 5.355470081704139e-15, 5.569688884972305e-15, 5.78390768824047e-15, 5.998126491508636e-15, 6.212345294776801e-15, 6.426564098044967e-15, 6.6407829013131324e-15], Int64[], 32, :s, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [24]:
FEM.showDoFResults(s2, :s)

7

In [25]:
err1 = FEM.fieldError(e1)

LowLevelFEM.TensorField(Matrix{Float64}[], [0.0 0.0 … 0.17362010953784177 0.738815085276597; 0.0 0.0 … 0.09220908167021089 0.3923434447109912; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 83295.87641762785 267685.95972034783], [0.0, 2.1421880326816568e-16, 4.2843760653633137e-16, 6.42656409804497e-16, 8.568752130726627e-16, 1.0710940163408283e-15, 1.285312819608994e-15, 1.4995316228771598e-15, 1.7137504261453255e-15, 1.927969229413491e-15  …  4.712813671899643e-15, 4.927032475167808e-15, 5.141251278435974e-15, 5.355470081704139e-15, 5.569688884972305e-15, 5.78390768824047e-15, 5.998126491508636e-15, 6.212345294776801e-15, 6.426564098044967e-15, 6.6407829013131324e-15], Int64[], 32, :e, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357))

In [26]:
FEM.showDoFResults(err1, :tensor)

8

In [27]:
u2, v2 = FEM.HHT(K, M, f, u0, v0, 10Tmin, Tmin / π)

(LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -3.5872029952351777e-28 … -3.4448715030461062e-9 -1.4886460525981895e-8; 0.0 5.7934801374396e-28 … 7.269168187948044e-9 3.144322500038513e-8], [0.0, 2.1421880326816568e-16, 4.2843760653633137e-16, 6.42656409804497e-16, 8.568752130726627e-16, 1.0710940163408283e-15, 1.285312819608994e-15, 1.4995316228771598e-15, 1.7137504261453255e-15, 1.927969229413491e-15  …  4.712813671899643e-15, 4.927032475167808e-15, 5.141251278435974e-15, 5.355470081704139e-15, 5.569688884972305e-15, 5.78390768824047e-15, 5.998126491508636e-15, 6.212345294776801e-15, 6.426564098044967e-15, 6.6407829013131324e-15], Int64[], 32, :u2D, LowLevelFEM.Problem("body1-2D", :PlaneStress, 2, 2, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 357)), LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0

In [28]:
FEM.showDoFResults(u2, :uvec)

9

In [29]:
FEM.showDoFResults(v2, :vvec)

10

In [30]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [31]:
gmsh.finalize()